In [171]:
import pandas as pd
import sys

In [2]:
df_raw = pd.read_csv('cdr.csv', sep=';', header=None)

In [3]:
df_raw.tail()

,0,1,2,3,4,5,6,7,8,9,10,11
1003771,Brasil Celular,2020-03-19 12:15:26,AndreiaTuttiDepil,5571986263634,Brasil Celular,30,AndreiaTuttiDepil,Tronco_TelleGroup_Cel,1,0,0.0325,0.3000
1003772,Brasil Fixo Sao Paulo,2020-03-19 12:14:52,Tronco_76Telecom,551146548130,Brasil Fixo Sao Paulo,117,Bytilly,\N,1,3,0.0000,0.0000
1003773,Brasil Fixo Sao Paulo,2020-03-19 12:14:10,brshield,551146422975,Brasil Fixo Sao Paulo,168,brshield,Tronco_Eai_Fixo_CLI,1,0,0.0420,0.0644
1003774,Brasil Fixo Sao Paulo,2020-03-19 12:15:24,Tronco_76Telecom,551139002562,Brasil Fixo Sao Paulo,109,Dellapar,\N,1,3,0.0000,0.0000
1003775,Brasil Celular,2020-03-19 12:18:06,AndreiaTuttiDepil,5585996759251,Brasil Celular,30,AndreiaTuttiDepil,Tronco_Eai_Cel_ITX,1,0,0.0325,0.3000


In [82]:
df_raw1 = df_raw[[0,1,2,3,5,6,7,10,11]].copy()
df_raw1.columns = ['Destino', 'Data', 'Conta Voip', 'Numero', 'Duracao', 'Usuario', 'Tronco','Compra', 'Venda']
df_raw1.head()

,Destino,Data,Conta Voip,Numero,Duracao,Usuario,Tronco,Compra,Venda
0,Brasil Fixo Sao Paulo,2016-12-31 18:02:31,1139001702,551142150756,30,\N,\N,0.0125,0.075
1,Brasil Celular,2016-12-31 18:46:22,1139001702,5511969519321,30,\N,Tronco_BFT_Cel,0.0900,0.400
2,Brasil Celular,2016-12-31 18:48:13,1139001702,5511967002466,36,\N,\N,0.1080,0.480
3,Brasil Celular,2016-12-31 19:20:27,1135265827,5511999718888,30,Rose_Alvorada,\N,0.0900,0.400
4,Brasil Celular,2016-12-31 19:25:08,1135265827,5511981371208,30,Rose_Alvorada,\N,0.0900,0.400


In [73]:
def split_tronco(str):
    lst = str.split('_')
    if len(lst) <3:
        return 'Entrada'
    
    if lst[2] == 'Cel':
        return 'Movel'
    
    elif lst[2] == 'Fixo':
        return 'Fixo'
    
    else:
        return 'outro'
df_raw1['Tipo_Ligação'] = df_raw1.Tronco.apply(split_tronco)
df_raw1.tail()

,Destino,Data,Conta Voip,Numero,Duracao,Usuario,Tronco,Compra,Venda,Tipo_Ligação
1003771,Brasil Celular,2020-03-19 12:15:26,AndreiaTuttiDepil,5571986263634,30,AndreiaTuttiDepil,Tronco_TelleGroup_Cel,0.0325,0.3000,Movel
1003772,Brasil Fixo Sao Paulo,2020-03-19 12:14:52,Tronco_76Telecom,551146548130,117,Bytilly,\N,0.0000,0.0000,Entrada
1003773,Brasil Fixo Sao Paulo,2020-03-19 12:14:10,brshield,551146422975,168,brshield,Tronco_Eai_Fixo_CLI,0.0420,0.0644,Fixo
1003774,Brasil Fixo Sao Paulo,2020-03-19 12:15:24,Tronco_76Telecom,551139002562,109,Dellapar,\N,0.0000,0.0000,Entrada
1003775,Brasil Celular,2020-03-19 12:18:06,AndreiaTuttiDepil,5585996759251,30,AndreiaTuttiDepil,Tronco_Eai_Cel_ITX,0.0325,0.3000,Movel


In [126]:
def busca_cliente(str):
    cli = df_raw1[df_raw1['Usuario'] == str]
    return cli

In [127]:
brshield = busca_cliente('brshield')

In [151]:
def cal_lucro(str):
    c = str.Compra.sum()
    v = str.Venda.sum()
    l = str.Venda.sum() - str.Compra.sum()
    print('''Somatoria dos valores :
    -> Compra R$ {:.2f} 
    -> Venda R$ {:.2f}
    -> Lucro R$ {:.2f}
    '''.format(c,v,l))

In [152]:
cal_lucro(brshield)

Somatoria dos valores :
    -> Compra R$ 7022.40 
    -> Venda R$ 9191.40
    -> Lucro R$ 2169.00
    


In [139]:
cal_lucro(brshield[selecao])

Somatoria dos valores :
    -> Compra R$ 496.82 
    -> Venda R$ 602.46
    -> Lucro R$ 105.64


In [174]:
def cal_clientes(df,i,f):
    '''
    Funcçao usada para calcular valores de custo, venda e lucro de uma tabela
    df -> Dataframe a ser analisado
    i -> Data inicio da busca (ex: '2019-01-01' ou '2019' ou '2019-01-01 00:00:00')
    f - Data final da busca (ex: '2019-01-01' ou '2019' ou '2019-01-01 00:00:00')
    '''
    cli = df['Usuario'].unique()
    p = (df['Data'] >= i) & (df['Data'] <= f)
    for c in cli :
        _df = df[df['Usuario'] == c]
        print(f'O cliente {c} no periodo de {i} ate {f} : ')
        cal_lucro(_df[p])

In [181]:
cal_clientes(df_raw1[df_raw1["Usuario"] == 'Escritorio'],'2019-01-01','2020-01-01')

O cliente Escritorio no periodo de 2019-01-01 ate 2020-01-01 : 
Somatoria dos valores :
    -> Compra R$ 239.42 
    -> Venda R$ 982.45
    -> Lucro R$ 743.03
    


In [180]:
cal_clientes(df_raw1,'2019','2020')

O cliente \N no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 8.33 
    -> Venda R$ 46.14
    -> Lucro R$ 37.81
    
O cliente Rose_Alvorada no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Omega_Metalurgica no periodo de 2019 ate 2020 : 


/media/carlos/Dados/projetos_python/plataforma_RVR/env/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


Somatoria dos valores :
    -> Compra R$ 1315.59 
    -> Venda R$ 6186.72
    -> Lucro R$ 4871.13
    
O cliente Escritorio no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 239.42 
    -> Venda R$ 982.45
    -> Lucro R$ 743.03
    
O cliente DY_Transportes no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Loma_Plast no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 534.92 
    -> Venda R$ 2310.90
    -> Lucro R$ 1775.99
    
O cliente Viviane_ArujaClub no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 8.67 
    -> Venda R$ 61.19
    -> Lucro R$ 52.52
    
O cliente Minercal no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 134.35 
    -> Venda R$ 706.39
    -> Lucro R$ 572.04
    
O cliente Bytilly no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 583.96 
    -> Venda R$ 3023.58
    -> Lucro R

O cliente Sthylluz_Reparos no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 118.61 
    -> Venda R$ 508.15
    -> Lucro R$ 389.54
    
O cliente Estronda_Calcados no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Voip_Link no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Nelson_Luiz no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Komandant_Parts no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente Pem_Transportes2 no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 0.00 
    -> Venda R$ 0.00
    -> Lucro R$ 0.00
    
O cliente brshield no periodo de 2019 ate 2020 : 
Somatoria dos valores :
    -> Compra R$ 530